Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Introduction to Azure Machine Learning: Deploy web service

In previous example, you ran an experiment to estimate value of pi. In this example, we'll use your estimated value to create a web service that computes the area of a circle in a real time. You'll learn about following concepts:

**Model** is simply a file - or a folder of files - that model management service tracks and versions. Typically a model file would contain the coefficients of your trained model, saved to a file.  

**Image** is a combination of model, a Python scoring script that tells how to load and invoke the model, and Python libraries used to execute that code. It is a self-contained unit that can be deployed as a service.

**Service** is the image running on a compute. The service can be called from your front-end application to get predictions, either using the Azure ML SDK or raw HTTP.

**Important**: This notebook uses Azure Container Instances (ACI) as the compute for the service. If you haven't registered ACI provider with your Azure subscription, run the following 2 cells first. Note that you must be the administrator of your Azure subscription to register a provider.

In [3]:
!az provider show -n Microsoft.ContainerInstance -o table

Namespace                    RegistrationState
---------------------------  -------------------
Microsoft.ContainerInstance  Registered


In [2]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FNLZ8CYYS to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "id": "7be2acd8-e2c7-4ec2-9ee7-697b22850d1e",
    "isDefault": true,
    "name": "Visual Studio Enterprise – MPN",
    "state": "Enabled",
    "tenantId": "76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61",
    "user": {
      "name": "swati.bannore@capgemini.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "76311cfb-5a24-4bc3-814a-2276543ff354",
    "isDefault": false,
    "name": "Sogeti India MS Practice",
    "state": "Enabled",
    "tenantId": "5c818d14-bdf7-4b8f-8b04-847c17d80e87",
    "user": {
      "name": "swati.bannore@capgemini.com",
      "type": "user"
    }
  }
]


In [4]:
!az provider register -n Microsoft.ContainerInstance

Let's load the workspace, and retrieve the latest run from your experiment using *Experiment.get_runs* method.

In [2]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle, json

In [3]:
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FKQ37BFNG to authenticate.
Interactive authentication successfully completed.


In [4]:
experiment_name = "azureml-experiment"
experiment = Experiment(workspace = ws, name = "azureml-experiment")
#run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]
experiment

Name,Workspace,Report Page,Docs Page
azureml-experiment,Azure_ML_Deployment,Link to Azure Portal,Link to Documentation


In [15]:
run = list(Experiment(workspace = ws, name = "my-first-experiment").get_runs())[0]

In [16]:
run.upload_file(name = 'outputs/model.pb', path_or_stream = './model.pb')

In the previous example you saved a file containing the pi value into run history. Registering the file makes it into a model that is tracked by Azure ML model management.

In [16]:
model = run.register_model(model_name = "cvmodel", model_path = "outputs\model.pb")

Let's create a scoring script that computes an area of a circle, given the estimate within the pi_estimate model. The scoring script consists of two parts: 

 * The *init* method that loads the model. You can retrieve registered model using *Model.get_model_path* method. 
 * The *run* method that gets invoked when you call the web service. It computes the area of a circle using the well-known $area = \pi*radius^2$ formula. The inputs and outputs are passed as json-formatted strings.

In [17]:
%%writefile score.py
from predict import initialize, predict_url, predict_image
from PIL import Image
import numpy as np

import base64
import io
import json

# Azure ML model loader
def init():
    initialize()

# Helper to predict an image encoded as base64
def predict_image_base64(encoded_image):
    if encoded_image.startswith('b\''):
        encoded_image = encoded_image[2:-1]

    decoded_img = base64.b64decode(encoded_image.encode('utf-8'))
    img_buffer  = io.BytesIO(decoded_img)

    image = Image.open(io.BytesIO(decoded_img))
    return predict_image(image)

# Azure ML entry point
def run(json_input):
    try:
        results = None
        input = json.loads(json_input)
        url = input.get("url", None)
        image = input.get("image", None)

        if url:
            results = predict_url(url)
        elif image:
            results = predict_image_base64(image)
        else:
            raise Exception("Invalid input. Expected url or image")
        return (results)
    except Exception as e:
        return (str(e))

Overwriting score.py


You also need to specify the library dependencies of your scoring script as conda yml file. This example doesn't use any special libraries, so let's simply use Azure ML's default dependencies.

In [18]:
from azureml.core.conda_dependencies import CondaDependencies 

cd = CondaDependencies()
cd.save_to_file(".", "myenv.yml")

'myenv.yml'

Then, let's deploy the web service on Azure Container Instance: a serverless compute for running Docker images. Azure ML service takes care of packaging your model, scoring script and dependencies into Docker image and deploying it.

In [20]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

# Define the configuration of compute: ACI with 1 cpu core and 1 gb of memory.
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Specify the configuration of image: scoring script, Python runtime (PySpark is the other option), and conda file of library dependencies.
image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                    runtime = "python", 
                                    conda_file = "myenv.yml")

# Deploy the web service as an image containing the registered model.
service = Webservice.deploy_from_model(name = "cv-webapi",
                                       deployment_config = aci_config,
                                       models = [model],
                                       image_config = image_config,
                                       workspace = ws)

# The service deployment can take several minutes: wait for completion.
service.wait_for_deployment(show_output = True)

Creating image
Running...................................
Succeeded
Image creation operation finished for image cv-webapi:1, operation "Succeeded"
Creating service
Running.........................
Failed

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: cv-webapi. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image azuremldeploym7735310446.azurecr.io/cv-webapi:1 locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: cv-webapi. From the AML SDK, you can run print(service.get_logs()) if you hav

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: cv-webapi. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image azuremldeploym7735310446.azurecr.io/cv-webapi:1 locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: cv-webapi. From the AML SDK, you 

In [8]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

In [25]:

# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/7be2acd8-e2c7-4ec2-9ee7-697b22850d1e/resourcegroups/AILabsTest/providers/Microsoft.ContainerService/managedClusters/aks-cv-compute466392968'

create_name='aks-cv-compute2' 
# Create the cluster
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# Wait for the operation to complete
aks_target.wait_for_completion(True)


Creating.........................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [11]:
#create_name='aks-cv-config' 
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'aks-cv-1' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [26]:
%%time
#aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Succeeded
None
CPU times: user 0 ns, sys: 2.53 ms, total: 2.53 ms
Wall time: 1.73 ms


In [27]:
aks_config = AksWebservice.deploy_configuration()

In [20]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with cv model",
                                                  tags = {'area': "cv", 'type': "cv"}
                                                 )

image = ContainerImage.create(name = "cvimage1",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running................................
Succeeded
Image creation operation finished for image cvimage1:1, operation "Succeeded"


In [28]:
%%time
aks_service_name ='aks-cv-api'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
print(aks_service.get_logs())

Creating service
Running........
Failed

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aks-cv-api. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image azuremldeploym7735310446.azurecr.io/cvimage1:1 locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed"

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aks-cv-api. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image azuremldeploym7735310446.azurecr.io/cvimage1:1 locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service st

You can try out the web service by passing in data as json-formatted request. Run the cell below and move the slider around to see real-time responses.

In [12]:
from ipywidgets import interact

def get_area(radius):
    request = json.dumps({"radius": radius})
    response = service.run(input_data = request)
    return json.loads(response)["area"]

interact(get_area,radius=(0,10))

interactive(children=(IntSlider(value=5, description='radius', max=10), Output()), _dom_classes=('widget-inter…

<function __main__.get_area(radius)>

Finally, delete the web service once you're done, so it's not consuming resources.

In [ ]:
service.delete()

As your next step, take a look at the more detailed tutorial for building an image classification model using Azure Machine Learning service.

[tutorials/img-classification-part1-training](./tutorials/img-classification-part1-training.ipynb)